***Initialization***

In [1]:
#Install PySpark
!pip install pyspark

In [2]:
#Import necessary libraries
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, avg

In [3]:
# Initialize Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
# Load CSV files into Spark DataFrames
df_games = spark.read.csv("nflgames.csv", header=True, inferSchema=True)
df_players = spark.read.csv("nflplayers.csv", header=True, inferSchema=True)
df_positions = spark.read.csv("nflpositions.csv", header=True, inferSchema=True)
df_teams = spark.read.csv("nflteams.csv", header=True, inferSchema=True)

The above code performs several tasks related to setting up a PySpark environment, initializing a Spark session, and loading CSV files into Spark DataFrames. Here's a description of each part of the code:

1. Install PySpark: This line installs the PySpark library using the pip package manager. PySpark is a Python library for Apache Spark, a distributed data processing framework.

2. Import necessary libraries: This section imports the required libraries for working with PySpark and data manipulation. It includes:
   - `SparkSession` from `pyspark.sql`: This is used to create and manage a Spark session.
   - `pandas` as `pd`: Pandas is a popular library for data manipulation and can be used alongside PySpark.
   - `col` and `avg` from `pyspark.sql.functions`: These functions are used for column operations and calculating the average, respectively.

3. Initialize Spark session: This code initializes a Spark session using the `SparkSession.builder.master("local[*]").getOrCreate()` method. It creates a Spark session with the specified configuration. In this case, `"local[*]"` means Spark will run in local mode using all available CPU cores.

4. Load CSV files into Spark DataFrames: This section reads CSV files (`nflgames.csv`, `nflplayers.csv`, `nflpositions.csv`, and `nflteams.csv`) into Spark DataFrames. The `spark.read.csv` method is used for this purpose. The `header=True` argument indicates that the first row of each CSV file contains column headers, and `inferSchema=True` attempts to automatically infer the data types of the columns.

Overall, this code prepares the environment for data analysis using PySpark by setting up a Spark session and loading the necessary data into DataFrames for further processing.

***SUMMARY OF THE DATASET***

In [5]:
# Display schema and sample data
df_games.printSchema()
df_games.show(5)

root
 |-- GameID: integer (nullable = true)
 |-- Week: integer (nullable = true)
 |-- HomeTeamID: integer (nullable = true)
 |-- AwayTeamID: integer (nullable = true)
 |-- HomeScore: integer (nullable = true)
 |-- AwayScore: integer (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- TimeOfDay: string (nullable = true)
 |-- FieldType: string (nullable = true)
 |-- Temp: integer (nullable = true)
 |-- Wind: integer (nullable = true)

+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+
|GameID|Week|HomeTeamID|AwayTeamID|HomeScore|AwayScore|DayOfWeek|TimeOfDay| FieldType|Temp|Wind|
+------+----+----------+----------+---------+---------+---------+---------+----------+----+----+
|     1|   1|         1|        29|       16|       23|      Sun|      Day| sportturf|  74|   8|
|     2|   2|         1|        28|       26|        6|      Thu|    Night| sportturf|  82|   6|
|     3|   3|        27|         1|       21|       23|      Sun| 

In [6]:
df_positions.printSchema()
df_positions.show(5)

root
 |-- PositionID: integer (nullable = true)
 |-- PositionName: string (nullable = true)

+----------+------------+
|PositionID|PositionName|
+----------+------------+
|         1|          QB|
|         2|          RB|
|         3|          WR|
|         4|          TE|
|         5|           K|
+----------+------------+



In [7]:
df_players.printSchema()
df_players.show(5)

root
 |-- PlayerID: integer (nullable = true)
 |-- PlayerName: string (nullable = true)
 |-- PositionID: integer (nullable = true)
 |-- TeamID: integer (nullable = true)

+--------+-----------------+----------+------+
|PlayerID|       PlayerName|PositionID|TeamID|
+--------+-----------------+----------+------+
|       1|      Kamar Aiken|         3|     1|
|       2|     Marlon Brown|         3|     1|
|       3|    Jeremy Butler|         3|     1|
|       4|Michael Campanaro|         3|     1|
|       5|     Owen Daniels|         4|     1|
+--------+-----------------+----------+------+
only showing top 5 rows



In [8]:
df_teams.printSchema()
df_teams.show(5)

root
 |-- TeamID: integer (nullable = true)
 |-- TeamName: string (nullable = true)
 |-- TeamCapsAbrv: string (nullable = true)
 |-- TeamAbrv: string (nullable = true)

+------+-------------------+------------+--------+
|TeamID|           TeamName|TeamCapsAbrv|TeamAbrv|
+------+-------------------+------------+--------+
|     1|   Baltimore Ravens|         RAV|     rav|
|     2|     Denver Broncos|         DEN|     den|
|     3|    Oakland Raiders|         RAI|     rai|
|     4|Philadelphia Eagles|         PHI|     phi|
|     5|     Dallas Cowboys|         DAL|     dal|
+------+-------------------+------------+--------+
only showing top 5 rows



This code displays the schema and a sample of the data from four Spark DataFrames: `df_games`, `df_positions`, `df_players`, and `df_teams`. It helps users understand the structure and contents of these DataFrames, which contain data related to NFL games, player positions, players, and teams.

 ***Team that has the highest average AwayScore as the Away Team***

In [9]:
# Initialize Spark session
spark = SparkSession.builder.appName("NFL_HighestAvgAwayScore").getOrCreate()

In [10]:
# Calculate the average AwayScore for each Away Team
avg_away_scores = df_games.groupBy("AwayTeamID").agg(avg("AwayScore").alias("AvgAwayScore"))

# Join df_teams to get the team names for Away Teams
avg_away_scores_with_names = avg_away_scores.join(df_teams, avg_away_scores["AwayTeamID"] == df_teams["TeamID"], "inner")

# Find the team with the highest average AwayScore
highest_avg_away_team = avg_away_scores_with_names.orderBy("AvgAwayScore", ascending=False).first()

In [11]:
# Display the result
print("Team with the Highest Average AwayScore as the Away Team:")
print("Team Name:", highest_avg_away_team["TeamName"])
print("Average AwayScore:", highest_avg_away_team["AvgAwayScore"])

Team with the Highest Average AwayScore as the Away Team:
Team Name: Indianapolis Colts
Average AwayScore: 35.0


The above code calculates and identifies the NFL team with the highest average "AwayScore" when playing as the away team in games.

1. Initializes a Spark session with the name "NFL_HighestAvgAwayScore."
2. Groups the `df_games` DataFrame by "AwayTeamID" and calculates the average of the "AwayScore" for each away team. The result is stored in the DataFrame `avg_away_scores` with the column name "AvgAwayScore."
3. Joins the `avg_away_scores` DataFrame with the `df_teams` DataFrame using the "AwayTeamID" as the joining key. This step associates team names with the calculated average away scores, creating a new DataFrame called `avg_away_scores_with_names`.
4. Orders the `avg_away_scores_with_names` DataFrame in descending order based on the "AvgAwayScore" column, effectively sorting teams by their average away scores.
5. Retrieves the team with the highest average away score by using `.first()` to get the top row of the sorted DataFrame. The result is stored in the `highest_avg_away_team` variable.
6. Prints the team with the highest average away score along with their team name and the average away score.

***Highest and Lowest Average Total Score***

In [12]:
# Calculate the total scores (HomeScore + AwayScore) for each game
df_games_with_totals = df_games.withColumn("TotalScore", col("HomeScore") + col("AwayScore"))

# Group the data by FieldType and calculate the average total score
avg_total_scores_by_fieldtype = df_games_with_totals.groupBy("FieldType").agg(avg("TotalScore").alias("AvgTotalScore"))

# Find the FieldType with the highest average total score
highest_avg_total_score = avg_total_scores_by_fieldtype.orderBy("AvgTotalScore", ascending=False).first()

# Find the FieldType with the lowest average total score
lowest_avg_total_score = avg_total_scores_by_fieldtype.orderBy("AvgTotalScore").first()

In [13]:
# Display the results
print("FieldType with the Highest Average Total Score:")
print("FieldType:", highest_avg_total_score["FieldType"])
print("Average Total Score:", highest_avg_total_score["AvgTotalScore"])

print("\nFieldType with the Lowest Average Total Score:")
print("FieldType:", lowest_avg_total_score["FieldType"])
print("Average Total Score:", lowest_avg_total_score["AvgTotalScore"])

FieldType with the Highest Average Total Score:
FieldType: astroplay
Average Total Score: 48.0

FieldType with the Lowest Average Total Score:
FieldType: grass
Average Total Score: 34.833333333333336


The above code calculates and identifies the NFL game field type (FieldType) with the highest and lowest average total scores (HomeScore + AwayScore). Here's a description of what the code does:

1. Adds a new column to the `df_games` DataFrame called "TotalScore," which represents the total points scored in each game. This is computed by summing the "HomeScore" and "AwayScore" columns.
2. Groups the `df_games_with_totals` DataFrame by the "FieldType" column and calculates the average total score for each field type. The results are stored in a new DataFrame called `avg_total_scores_by_fieldtype`, with the calculated average total score labeled as "AvgTotalScore."
3. Orders the `avg_total_scores_by_fieldtype` DataFrame in descending order based on the "AvgTotalScore" column to find the field type with the highest average total score. The result is stored in the `highest_avg_total_score` variable.
4. Orders the `avg_total_scores_by_fieldtype` DataFrame in ascending order based on the "AvgTotalScore" column to find the field type with the lowest average total score. The result is stored in the `lowest_avg_total_score` variable.
5. Displays the results, including the field type with the highest and lowest average total scores, along with their respective average total scores.


***Average Players Per Position***

In [14]:
# Calculate the average number of players in each position for each team
average_players_per_position = df_players.groupBy("TeamID", "PositionID").count() \
    .groupBy("TeamID").pivot("PositionID").agg(avg("count"))

In [15]:
# Show the result
average_players_per_position.show()

+------+---+---+----+---+----+
|TeamID|  1|  2|   3|  4|   5|
+------+---+---+----+---+----+
|    31|3.0|3.0| 7.0|4.0| 1.0|
|    28|3.0|2.0| 7.0|3.0| 1.0|
|    27|2.0|3.0| 7.0|4.0| 1.0|
|    26|3.0|5.0| 8.0|4.0| 2.0|
|    12|3.0|5.0| 8.0|4.0| 1.0|
|    22|2.0|4.0| 8.0|3.0| 1.0|
|     1|2.0|3.0| 7.0|3.0| 1.0|
|    13|4.0|3.0| 7.0|4.0| 1.0|
|    16|3.0|4.0| 9.0|5.0| 1.0|
|     6|2.0|4.0|11.0|5.0| 1.0|
|     3|3.0|6.0| 7.0|4.0| 1.0|
|    20|3.0|4.0| 5.0|2.0| 2.0|
|     5|3.0|3.0| 5.0|4.0| 1.0|
|    19|2.0|4.0| 5.0|5.0| 1.0|
|    15|3.0|4.0| 5.0|4.0|NULL|
|    17|4.0|6.0| 5.0|6.0| 1.0|
|     9|2.0|6.0| 7.0|4.0| 1.0|
|     4|4.0|3.0| 5.0|4.0| 1.0|
|     8|3.0|3.0| 9.0|2.0| 1.0|
|    23|2.0|6.0| 7.0|6.0| 1.0|
+------+---+---+----+---+----+
only showing top 20 rows



The above code calculates the average number of players in each position for each NFL team and displays the results.

1. It first groups the data by two columns: "TeamID" and "PositionID." This groups players by their respective teams and positions.

2. Then, it counts the number of players in each combination of team and position.

3. After counting the players in each position for each team, the code further groups the data by "TeamID." This step prepares the data for a pivot operation.

4. The `pivot` function is applied to the grouped data. It essentially transforms the rows into columns, with each unique position ID becoming a separate column. The pivot operation calculates the average count of players for each position within each team. This results in a DataFrame where rows represent teams, and columns represent different positions (e.g., QB, RB, WR, etc.), with the average player count in each position.

5. Finally, the code displays the calculated average player counts for each position within each NFL team using the `show()` function.


***Stopping the Spark Session***

In [16]:
spark.stop()

When spark.stop() is executed, it shuts down the Spark application gracefully. This means that any ongoing Spark jobs or tasks will be halted, and Spark resources will be released.